# What's the flow rate through JdF in the months we were comparing to CIOPS?

In [1]:
import xarray as xr
import numpy as np
import datetime
import matplotlib.pyplot as plt
from pathlib import Path
from lo_tools import plotting_functions as lo_plot
from datetime import timedelta

In [2]:
grid = xr.open_dataset('/data1/parker/LO_data/grids/cas6/grid.nc')

x = 316
ymin = 897   
ymax = 946

In [3]:
def get_his_fn_from_dt(dt):

    path = Path("/data1/parker/LO_roms")
    # This creates the Path of a history file from its datetime
    if dt.hour == 0:
        # perfect restart does not write the 0001 file
        dt = dt - timedelta(days=1)
        his_num = '0025'
    else:
        his_num = ('0000' + str(dt.hour + 1))[-4:]
    date_string = dt.strftime('%Y.%m.%d')
    fn = path / 'cas6_v0_live' / ('f' + date_string) / ('ocean_his_' + his_num + '.nc')
    return fn

In [4]:
days = 30
# for the 2018 winter run
start = datetime.datetime(2018,2,7)
dates = [start + timedelta(hours=i) for i in range(days*24)]
fn = [get_his_fn_from_dt(date) for date in dates]

In [18]:
flow = 0

for f in fn:

    dist = lo_plot.get_sect(f,'u',np.repeat(x,(ymax+1)-(ymin-1)),np.arange(ymin-1,ymax+1))[6]
    z = lo_plot.get_zfull(0,f,'u')[:,ymin:ymax+1,x]
    
    width = dist[:,1:]-dist[:,:-1]
    height = (z[:-1,:]-z[1:,:])*-1

    u = xr.open_dataset(f).u.sel(eta_u=slice(ymin, ymax+1), xi_u=x).squeeze(dim='ocean_time')

    flow += np.sum(np.where(u>0, u, 0)*height[:-1]*width[:-1])

In [21]:
flow/(len(dates))

127863.08407233242